<a href="https://colab.research.google.com/github/Binaaa04/magang_DataScience/blob/main/Data_Sains_Sabrina_Rahmadini_Tugas_Modul_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# === Load dataset ===
df = pd.read_csv('/content/drive/MyDrive/MAGANG/DATA SCIENCE/dataSalesCafeClean.csv', low_memory=False)

# Lihat struktur data
df.info()

# Lihat 5 baris pertama
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    10000 non-null  object 
 1   Item              10000 non-null  object 
 2   Quantity          10000 non-null  object 
 3   Price Per Unit    10000 non-null  float64
 4   Total Spent       10000 non-null  float64
 5   Payment Method    10000 non-null  object 
 6   Location          10000 non-null  object 
 7   Transaction Date  10000 non-null  object 
 8   Log_Total_Spent   10000 non-null  float64
dtypes: float64(3), object(6)
memory usage: 703.3+ KB


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date,Log_Total_Spent
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08,1.609438
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16,2.564949
2,TXN_4271903,Cookie,4,1.0,8.0,Credit Card,In-store,2023-07-19,2.197225
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27,2.397895
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11,1.609438


In [ ]:
import pandas as pd
import plotly.express as px
from scipy.stats import ttest_ind

# === Load dataset ===
df = pd.read_csv('/content/drive/MyDrive/MAGANG/DATA SCIENCE/dataSalesCafeClean.csv', low_memory=False)

# =========================
# Produk paling banyak dibeli
# =========================
top_product = df['Item'].value_counts().reset_index()
top_product.columns = ['Item', 'Total']

fig = px.bar(
    top_product,
    x='Item',
    y='Total',
    title='Produk yang Paling Banyak Dibeli'
)
fig.show()

# =========================
# Metode Pembayaran
# =========================
payment = df['Payment Method'].value_counts().reset_index()
payment.columns = ['Payment Method', 'Count']

fig = px.pie(
    payment,
    names='Payment Method',
    values='Count',
    title='Persentase Metode Pembayaran'
)
fig.show()

# =========================
#  Aktivitas Penjualan Harian
# =========================
daily_sales = df.groupby('Transaction Date').size().reset_index(name='Transactions')

fig = px.line(
    daily_sales,
    x='Transaction Date',
    y='Transactions',
    title='Aktivitas Penjualan Harian'
)
fig.show()

# =========================
# ANALISA PERTANYAAN 1
# Apakah rata-rata penjualan hari kerja lebih tinggi dibanding akhir pekan?
# =========================
# Ubah kolom tanggal ke format datetime
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])
df['day'] = df['Transaction Date'].dt.day_name()

# Pastikan kolom Total Spent bertipe numerik
df['Total Spent'] = pd.to_numeric(df['Total Spent'], errors='coerce')

# Kelompokkan hari kerja & akhir pekan, hapus nilai NaN
weekday = df[df['day'].isin(['Monday','Tuesday','Wednesday','Thursday','Friday'])]['Total Spent'].dropna()
weekend = df[df['day'].isin(['Saturday','Sunday'])]['Total Spent'].dropna()

# Uji-t
tstat, pval = ttest_ind(weekday, weekend, equal_var=False)
print("Rata-rata Hari Kerja:", weekday.mean())
print("Rata-rata Akhir Pekan:", weekend.mean())
print("T-statistik:", tstat)
print("P-value:", pval)

# Visualisasi perbandingan rata-rata
avg_weekday = weekday.mean()
avg_weekend = weekend.mean()

avg_df = pd.DataFrame({
    'Kategori': ['Hari Kerja', 'Akhir Pekan'],
    'Rata-rata Transaksi': [avg_weekday, avg_weekend]
})

fig = px.bar(
    avg_df,
    x='Kategori',
    y='Rata-rata Transaksi',
    title='Perbandingan Rata-rata Penjualan: Hari Kerja vs Akhir Pekan',
    text='Rata-rata Transaksi'
)
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.show()

# =========================
# ANALISA PERTANYAAN 2
# Apakah rata-rata transaksi produk minuman (Juice, Coffee) lebih tinggi dibanding produk makanan (Cake, Sandwich, Salad)?
# =========================
# Kelompokkan produk jadi dua kategori, hapus NaN
minuman = df[df['Item'].isin(['Juice', 'Coffee'])]['Total Spent'].dropna()
makanan = df[df['Item'].isin(['Cake', 'Sandwich', 'Salad'])]['Total Spent'].dropna()

# Uji-t dua sampel independen
t_stat, p_val = ttest_ind(minuman, makanan, equal_var=False)

# Cetak hasil uji statistik
print("\n=== Analisis Minuman vs Makanan ===")
print("T-statistik:", t_stat)
print("P-value:", p_val)
print("Rata-rata Transaksi Minuman:", minuman.mean())
print("Rata-rata Transaksi Makanan:", makanan.mean())

# Interpretasi hasil uji statistik (perbaikan di sini)
if p_val < 0.05:
    if minuman.mean() > makanan.mean():
        print("Hasilnya signifikan → Rata-rata transaksi **minuman** lebih tinggi daripada **makanan**.")
    else:
        print("Hasilnya signifikan → Rata-rata transaksi **makanan** lebih tinggi daripada **minuman**.")
else:
    print("Tidak signifikan → Rata-rata transaksi minuman dan makanan tidak berbeda jauh.")

# Visualisasi Rata-rata Transaksi per Kelompok
avg_values = {
    'Kategori': ['Minuman', 'Makanan'],
    'Rata-rata Transaksi': [minuman.mean(), makanan.mean()]
}

avg_df = pd.DataFrame(avg_values)

fig = px.bar(
    avg_df,
    x='Kategori',
    y='Rata-rata Transaksi',
    title='Perbandingan Rata-rata Transaksi: Minuman vs Makanan',
    text='Rata-rata Transaksi'
)
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.show()


Rata-rata Hari Kerja: 8.877683034401647
Rata-rata Akhir Pekan: 8.878517823639775
T-statistik: -0.006623717025844904
P-value: 0.9947152897228745



=== Analisis Minuman vs Makanan ===
T-statistik: -31.6344056131309
P-value: 4.6817584386180627e-203
Rata-rata Transaksi Minuman: 7.730610715623829
Rata-rata Transaksi Makanan: 11.950263311878292
Hasilnya signifikan → Rata-rata transaksi **makanan** lebih tinggi daripada **minuman**.


Perbandingan Hasil Kerja vs Akhir Pekan :

Berdasarkan analisis data transaksi, tidak ditemukan perbedaan signifikan antara rata-rata penjualan pada hari kerja dan akhir pekan (p-value: 0.9947). Artinya, aktivitas penjualan cenderung stabil sepanjang minggu. Hal ini menunjukkan bahwa pengalokasian sumber daya operasional dapat dilakukan secara merata tanpa perlu fokus khusus pada akhir pekan. Selain itu, stabilitas ini membuka peluang untuk eksperimen promosi di hari-hari tertentu guna mengidentifikasi waktu terbaik untuk mendorong peningkatan transaksi.

Perbandingan Hasil Analisa Minuman vs Makanan :

Berdasarkan hasil analisis dan uji statistik, rata-rata transaksi untuk kategori makanan (11.95) secara signifikan lebih tinggi dibandingkan dengan minuman (7.73), dengan p-value yang sangat kecil (4.68e-203). Hal ini menunjukkan bahwa makanan merupakan kontributor utama dalam transaksi harian. Dalam konteks bisnis, insight ini mengindikasikan bahwa strategi promosi dan pengembangan produk sebaiknya lebih difokuskan pada kategori makanan. Namun, rendahnya performa kategori minuman membuka peluang untuk evaluasi produk dan strategi upselling, seperti bundling makanan-minuman guna mendorong pertumbuhan dari sisi minuman.